In [1]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd
import pickle

import h5py
import re
import gc
import sys

import keras

from keras.models import Model, load_model
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, \
        BatchNormalization, GRU, LSTM
from keras.callbacks import EarlyStopping
from keras.preprocessing.sequence import pad_sequences

import time
import datetime

from sklearn.metrics import roc_curve, auc


from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from datetime import datetime

C:\dev\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
with open('..//datasets//joined_train_with_metro_data.pickle','rb') as f:
    df_features_train = pickle.load(f)
    
with open('..//datasets//joined_test_with_metro_data.pickle','rb') as f:
    df_features_test = pickle.load(f)

In [24]:
print(r)

asass 02


In [3]:
df_synoptic = pd.read_csv('synoptic_classification_1948-14_May_2017.csv')
df_synoptic = df_synoptic.loc[:,['Month','Day','2016','2017']]

In [25]:
unique_ordered_month = []
for word in df_synoptic['Month']:
    if word not in unique_ordered_month:
        unique_ordered_month.append(word)
Month_dict = dict(zip(unique_ordered_month, ['01','02','03','04','05','06','07','08','09','10','11','12']))
df_synoptic['Month'] = df_synoptic['Month'].apply(lambda x:Month_dict[x])
df_synoptic2016 = df_synoptic[['Month','Day','2016']].apply(lambda x: '%02d' % (x))

TypeError: ('%d format: a number is required, not Series', 'occurred at index Month')

In [7]:
df_features_train.columns

Index(['Validity date', 'Persist. value_tmax_1', 'EC_tmax_1', 'CO_tmax_1',
       'C3_tmax_1', 'OH_tmax_1', 'Persist. value_tmin_1', 'EC_tmin_1',
       'CO_tmin_1', 'C3_tmin_1', 'OH_tmin_1', 'Persist. value_wind_1',
       'EC_wind_1', 'CO_wind_1', 'C3_wind_1', 'OH_wind_1',
       'Persist. value_humidity_1', 'EC_humidity_1', 'C3_humidity_1',
       'Persist. value_tmax_2', 'EC_tmax_2', 'CO_tmax_2', 'C3_tmax_2',
       'OH_tmax_2', 'Persist. value_tmin_2', 'EC_tmin_2', 'CO_tmin_2',
       'C3_tmin_2', 'OH_tmin_2', 'Persist. value_wind_2', 'EC_wind_2',
       'CO_wind_2', 'C3_wind_2', 'OH_wind_2', 'Persist. value_humidity_2',
       'EC_humidity_2', 'C3_humidity_2', 'cityID', 'stationID', 'name', 'E',
       'N', 'elevation ', 'elevationWindMet', 'CoastalDist', 'archiveID',
       'City_0', 'City_1', 'City_2', 'City_3', 'City_4', 'City_5', 'City_6',
       'City_7', 'City_8', 'City_9', 'City_10', 'City_11', 'City_12',
       'City_13', 'City_14'],
      dtype='object')

In [25]:
feature_names_input = ['Persist. value_tmax_1', 'EC_tmax_1', 'CO_tmax_1',
       'C3_tmax_1', 'OH_tmax_1', 'Persist. value_tmin_1', 'EC_tmin_1',
       'CO_tmin_1', 'C3_tmin_1', 'OH_tmin_1', 'Persist. value_wind_1',
       'EC_wind_1', 'CO_wind_1', 'C3_wind_1', 'OH_wind_1',
       'Persist. value_humidity_1', 'EC_humidity_1', 'C3_humidity_1',
       'Persist. value_tmax_2', 'EC_tmax_2', 'CO_tmax_2', 'C3_tmax_2',
       'OH_tmax_2', 'Persist. value_tmin_2', 'EC_tmin_2', 'CO_tmin_2',
       'C3_tmin_2', 'OH_tmin_2', 'Persist. value_wind_2', 'EC_wind_2',
       'CO_wind_2', 'C3_wind_2', 'OH_wind_2', 'Persist. value_humidity_2',
       'EC_humidity_2', 'C3_humidity_2']

In [24]:
len(df_features_train)/30

364.5

In [16]:
df_features_train.iloc[range(0,10000,1000)]['archiveID']

0       5811
1000    3014
2000    7841
3000    9974
4000    9476
5000     850
6000    2523
7000    8229
8000    4741
9000    1980
Name: archiveID, dtype: int64

In [ ]:
def flatten_cities(df_per_city,feature_names_input):
    col_names = ['Validity date'] + feature_names_input
    for val_date in df_per_city['Validity date']:
        

In [18]:
df_synoptic.head(10)

,Month,Day,2016,2017
0,Jan,1,11,1.0
1,Jan,2,9,17.0
2,Jan,3,15,1.0
3,Jan,4,7,1.0
4,Jan,5,10,3.0
5,Jan,6,7,9.0
6,Jan,7,16,1.0
7,Jan,8,14,14.0
8,Jan,9,8,16.0
9,Jan,10,1,7.0


In [ ]:
# MLP for Pima Indians Dataset with 10-fold cross validation
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import StratifiedKFold
import numpy
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load pima indians dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []
for train, test in kfold.split(X, Y):
  # create model
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model
    model.fit(X[train], Y[train], epochs=150, batch_size=10, verbose=0)
    # evaluate the model
    scores = model.evaluate(X[test], Y[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))